In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
import string
import csv
import shutil
import copy
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
import spacy
from spacy import lookups
from spacy import tokenizer
import nltk
from nltk import word_tokenize
#from nltk import nlp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
import gensim.models
import pickle
import joblib
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import WordNetCorpusReader

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
#Creates the rules input data.
#Initializing text is the text of the Sentence
# be indicates which 'be' is being looked at in the sentence
# # num is the index of the line from the original csv
# for eg. if there are multiple 'be' in a sentence, the first will have be = 1, second will have be = 2, etc to distinguish them
class Sentence:
    habituality = 0 #using a ternary system: -1 = nonhabitual 0 = unclassified, 1 = habitual
    pos1 = 0; pos2 = 0; pos3 = 0; pos4 = 0; pos5 = 0; pos6 = 0;
    synPar1 = 0; synPar2 = 0; synPar3 = 0; synPar4 = 0; 
    a1 = 0; a2 = 0; a3 = 0; a4 = 0; a5 = 0; r1 = 0;

    def __init__(self, text, be, num):
        self.text = text
        self.be = be
        self.num = num

In [ ]:
lines = []

with open("gold standard sentences no augmentation.csv", "r", encoding="utf_8") as file:
    lines = file.readlines()
        
print(len(lines))

In [ ]:
#checks the numbers of 'be' and duplicates the sentence if there are more than one 'be' to the input data
sen = []
for line in range(len(lines)):
        be = 1
        numBe = 0
        parsed = nlp(lines[line])
        for i, word in enumerate(parsed):
            if (word.text.lower() == "be"):
                numBe += 1
        if (numBe >= 1):
            sen.append(Sentence(lines[line], be, line))
        while (numBe > 1):
            be += 1
            sen.append(Sentence(lines[line], be, line))
            numBe -= 1
for Sentence in range(len(sen)):
    sen[Sentence].text = (sen[Sentence].text).replace('"','')

In [ ]:
print(sen[1].text)
print(len(sen))

In [ ]:
phonetic_variation = ["wanna", "tryna", "gonna", "gotta"]  

In [ ]:
#Below code takes few words before and after 'be' and analyses these words using the rules (pos1-pos6),(synpar1-synpar6),(A1-A5)and (R1) and updates the specific columns with the same name
index = -1
senNum = 1
numBe = 0
extraBe = []
beIndices = []
count = 0

for Sentence in sen:
    index += 1 
    numBe = 0
    if (index > 0 and (sen[index].text == sen[index-1].text) and (sen[index].num == sen[index-1].num)):
        senNum += 1
    else:
        senNum = 1
    parsed = nlp(Sentence.text)
    for i, word in enumerate(parsed):
        if (word.text.lower() == "be"):
            numBe += 1
            #if (numBe == senNum):
            if (numBe == 1):
                print(Sentence.text)
                count +=1
                beIndices.append(i)
                beChildren = list(word.children)
                if(word.head == word):
                    beSibling = []
                else:
                    beSibling = list(word.head.children)
                beDep = word.dep_
                bePOS = word.pos_
                beIndex = i
                preprepreprevious = None
                prepreprevious = None
                preprevious = None
                previous = None
                after = None
                postafter = None
                postpostafter = None
                if(i >= 4):
                    preprepreprevious = parsed[i-4]
                if(i >= 3):
                    prepreprevious = parsed[i-3]
                if(i >= 2):
                    preprevious = parsed[i-2]
                if(i > 0):
                    previous = parsed[i-1]
                if(i+1 < len(parsed)):
                    after = parsed[i+1]
                if(i+2 < len(parsed)):
                    postafter = parsed[i+2]
                if(i+3 < len(parsed)):
                    postpostafter = parsed[i+3]
                #pos1
                if(previous and (previous.tag_ == "MD" or previous.tag_ == "JJ" or previous.tag_ == "TO")):
                    Sentence.pos1 = 1

                #pos1 part 2
                if(previous and previous.text in phonetic_variation):
                    Sentence.pos1 = 1

                #pos2
                if ((after and previous) and (after.tag_ == "JJ" and previous.tag_ != "PRP" and not previous.tag_.__contains__("NN"))):
                    Sentence.pos2 = 1

                #pos3
                if ((after and previous) and (after.tag_ == "IN" and (previous.tag_ == "VBZ" or previous.tag_ == "VBP"))):
                    Sentence.pos3 = 1

                #pos4
                if (previous and preprevious) and (previous.tag_ == "NN" and preprevious.tag_ == "JJ"):
                    Sentence.pos4 = 1

                #pos5
                if (previous and after) and (previous.tag_ == "RB" and (after.tag_ == "PRP" or after.tag_ == "DT")):
                    Sentence.pos5 = 1

                #pos6
                if (previous and preprevious) and (previous.tag_ == "RB" and preprevious.tag_.__contains__("VB") or preprevious.tag_ == "MD"):
                    Sentence.pos6 = 1

                #synPar1 -> aux_children:
                if len(beChildren) != 0:
                    for child in beChildren:
                        if child.dep_ == "aux" and child.pos_ == "AUX" and child.text.lower() != "do":
                            Sentence.synPar1 = 1
                #synPar2 -> aux_siblings
                if len(beSibling) != 0:
                    for sibling in beSibling:
                        if sibling.dep_ == "aux" and sibling.pos_ == "AUX" and sibling.text.lower() != "do" and sibling.text != "be":
                            Sentence.synPar2 = 1
                #synPar3 -> verbal_auxilary
                if (word.pos_ == "AUX" and word.dep_ == "aux" and word.head.pos_ == "VERB"):
                    Sentence.synPar3 = 1
                   
                #synPar4 -> copular_verb
                if(word.pos_ == "VERB"):
                    Sentence.synPar4 = 1
                #A1
                if (((previous and preprevious and prepreprevious) and (previous.text == "n't" and ((preprevious.text == "do") or (preprevious.text == "Do"))) and (prepreprevious.pos_ == "PRON" or prepreprevious.pos_ == "NOUN")) or ((preprevious and prepreprevious and preprepreprevious) and ((previous.pos_ != "VERB" and previous.pos_ != "AUX")) and preprevious.text == "n't" and ((prepreprevious.text == "do") or (prepreprevious.text == "Do")) and (preprepreprevious.pos_ == "PRON" or preprepreprevious.pos_ == "NOUN"))):
                    Sentence.a1 = 1

                #A2
                if (((after) and (after.pos_ == "PUNCT" or after.pos_ == "CCONJ" or after.pos_ == "DET" or after.pos_ == "INTJ" or after.pos_ == "PROPN"))):
                    Sentence.a2 = 1


                #A3

                if ((previous and previous.pos_ == "PRON") or (preprevious and preprevious.pos_ == "PRON" and previous.pos_ != "AUX" and previous.pos_ != "VERB" and previous.pos_ != "PART") or (prepreprevious and prepreprevious.pos_ == "PRON" and previous.pos_ == "ADV" and preprevious.pos_ != "AUX" and preprevious.pos_ != "VERB")):
                    Sentence.a3 = 1


                #A4

                if (after and after.tag_ == "VBG" and previous and previous.pos_ != "AUX" and not (previous.pos_ == "PART" and Sentence.pos1 == 1)):
                    Sentence.a4 = 1

            # if (after and after.tag_ == "VBG" and previous and previous.pos_ != "AUX" and not (previous.pos_ == "PART" and previous.tag_  == "TO" and not previous.text in phonetic_variation)):
                #'Be' is followed by a verb ending in 'ing' and is not preceded by an auxiliary verb, ‘to’, or any of the words in phonetic variation: ‘gonna’, ‘gotta’, ‘wanna’, or ‘tryna’; tends to be Habitual.    
                    
                #A5

                if ((previous and preprevious) and (previous.text == "n't" and preprevious.text != "do" and preprevious.text != "Do")):
                    Sentence.a5 = 1
                #R1

                if ((Sentence.pos1 !=1) and (Sentence.pos2 !=1) and (Sentence.pos3 !=1) and (Sentence.pos4 !=1) and (Sentence.pos5 !=1) and (Sentence.pos6 !=1)):
                    Sentence.r1 = 1
            else:
                numBe -= 1




In [ ]:
#Introduces the interaction between the rules based on text analysis
for Sentence in sen:
    if (Sentence.r1 == 0):
        Sentence.synPar3 = 0
    if (Sentence.a4 == 1):
        Sentence.synPar3 = 0
    if (Sentence.a2 == 1 and Sentence.a3 == 1):
        Sentence.pos5 = 0
    if (Sentence.a4 == 1):
        Sentence.a5 == 0
    if (Sentence.a3 == 1):
        Sentence.synPar2 = 0
    if (Sentence.pos6 == 1 or Sentence.synPar1 == 1):
        Sentence.a3 = 0
    if (Sentence.synPar1 == 1):
        Sentence.r1 = 0

In [ ]:
#creates spreadsheet document
with open("harrison synpar A-inter test rules coraal_analysis_spreadsheet.csv", "w", encoding="utf-8-sig", newline="") as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
quotechar='"', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(["Habituality", "Sentence", "POS1", "POS2", "POS3", "POS4", "POS5", "POS6","A1", "A2","A3", "A4", "A5", "SynPar1", "SynPar2", "SynPar3", "SynPar4", "R1"])
    for Sentence in sen:
        filewriter.writerow([Sentence.habituality, Sentence.text, Sentence.pos1, Sentence.pos2, Sentence.pos3, Sentence.pos4, Sentence.pos5,Sentence.pos6, Sentence.a1, Sentence.a2, Sentence.a3, Sentence.a4, Sentence.a5, Sentence.synPar1, Sentence.synPar2, Sentence.synPar3, Sentence.synPar4, Sentence.r1])
    

In [ ]:
# preData = pd.read_csv("harrison synpar test rules coraal_analysis_spreadsheet.csv")
#import chardet

# Detect the encoding of the file
#with open("harrison synpar A-inter test rules coraal_analysis_spreadsheet.csv", "rb") as f:
#    result = chardet.detect(f.read())

In [ ]:
#Reading the input data which has all the information on rules
preData = pd.read_csv("harrison synpar A-inter test rules coraal_analysis_spreadsheet.csv")
                      #, encoding='ISO-8859-1')

In [ ]:
print(preData)

In [ ]:
class Sentence:
    habituality = 0 #using a ternary system: -1 = nonhabitual 0 = unclassified, 1 = habitual
    pos1 = 0; pos2 = 0; pos3 = 0; pos4 = 0; pos5 = 0; pos6 = 0; s1 = 0; a2 = 0; a3 = 0; a4 = 0; a5 = 0; a6 = 0;
    synPar1 = 0; synPar2 = 0; synPar3 = 0; synPar4 = 0; r1 = 0;
    
    def __init__(self, text):
        self.text = text

In [ ]:
lines = []
with open("gold standard sentences no augmentation.csv", "r", encoding="utf_8") as file:
    lines = file.readlines()

i = 0
del lines[0]
while (i < len(lines)):
    if (lines[i] == "\"\n" or lines[i] == "\"" or lines[i] == ' "\n'):
        del lines[i]
    i +=1
print(len(lines))

In [ ]:
print(lines)

In [ ]:
sentences = []

for line in lines:
    if(line != '",,,,,\n'):
        sentences.append(Sentence(line))

In [ ]:
print(len(preData))
print(len(sentences))

In [ ]:
#code for handling Multiple 'be'
sen = []
beIndicies = []
numBe = 0
extraBe = []

data = copy.deepcopy(preData)
i = len(preData)-1
while (i >= 0):
    data.drop(i, inplace=True)
    i = i -1
    
print(len(data))
    
for Sentence in range(len(sentences)):
    numBe = 0
    parsed = nlp(sentences[Sentence].text)
    for i, word in enumerate(parsed):
        if (word.text.lower() == "be"):
            numBe += 1
    if (numBe >= 1):
#    if (numBe == 1):
        sen.append(sentences[Sentence])
    while (numBe > 1):
        sen.append(sentences[Sentence])
        numBe -= 1

print(len(sen))
        

In [ ]:
for Sentence in range(len(sen)):
    numBe = 0
    parsed = nlp(sen[Sentence].text)
    for i, word in enumerate(parsed):
        if (word.text.lower() == "be"):
            numBe += 1
    if (numBe >= 1): 
#    if (numBe == 1):
        data = data.append(preData.iloc[Sentence], ignore_index=True)


In [ ]:
print(data)

In [ ]:
w = []
pppp = []
ppp = []
pp = []
p = []
a = []
pa = []
ppa = []
pppa = []

In [ ]:
#appending part of speech of the words before and after 'be'
numBe = 1

for Sentence in sen:
    if (numBe == 1):
        numBe = 0
        parsed = nlp(Sentence.text)
        for i, word in enumerate(parsed):
            if (word.text.lower() == "be"):
                numBe += 1
                w.append(word.pos_)
                if(i >= 4):
                    preprepreprevious = parsed[i-4]
                    pppp.append(preprepreprevious.pos_)
                else:
                    pppp.append(-1)
                if(i >= 3):
                    prepreprevious = parsed[i-3]
                    ppp.append(prepreprevious.pos_)
                else:
                    ppp.append(-1)
                if(i >= 2):
                    preprevious = parsed[i-2]
                    pp.append(preprevious.pos_)
                else:
                    pp.append(-1)
                if(i > 0):
                    previous = parsed[i-1]
                    p.append(previous.pos_)
                else:
                    p.append(-1)
                if(i+1 < len(parsed)):
                    after = parsed[i+1]
                    a.append(after.pos_)
                else:
                    a.append(-1)
                if(i+2 < len(parsed)):
                    postafter = parsed[i+2]
                    pa.append(postafter.pos_)
                else:
                    pa.append(-1)
                if(i+3 < len(parsed)):
                    postpostafter = parsed[i+3]
                    ppa.append(postafter.pos_)
                else:
                    ppa.append(-1)
                if(i+4 < len(parsed)):
                    postpostpostafter = parsed[i+4]
                    pppa.append(postpostpostafter.pos_)
                else:
                    pppa.append(-1)
    else:
        numBe -= 1

In [ ]:
#data.rename(columns = {"ï»¿Habituality": "Habituality"}, inplace = True)

In [ ]:
## reading the file
#Creates unigrams-bigrams for 4 words before and after 'be'
#df = pd.read_csv('spam.csv',encoding = 'ISO-8859-1',usecols=['v1','v2'])
corpus = [] #empty list
wlemmatizer = WordNetLemmatizer()
length = len(data['Sentence']) #finding total number of rows
be_sen = []
for i in data['Sentence']:
    start = 0
    end = 0
    if nlp(i) != parsed:
        parsed = nlp(i)
        for j, word in enumerate(parsed):
            if word.text.lower() == "be":
                if(j >= 4):
                    start = j-4
                else:
                    start = 0
                if(len(i) > j+4):
                    end = j+4    
                else:
                    end = len(i)
        be_sen.append(parsed[start:end])

print(len(be_sen))
for i in range(length):
	rev = re.sub('[^a-zA-Z]',' ',str(be_sen[i]))
	rev = rev.lower() #text to lowercase
	rev = rev.split() #each word of the sentence becomes the element of a list
	rev = [wlemmatizer.lemmatize(word) for word in rev if word not in stopwords.words('english')] #lemmatization via list comprehension
	rev = ' '.join(rev) #from list to string
	corpus.append(rev) #appending to the list
    
#ngram_range=(3, 3)
cv = CountVectorizer(ngram_range=(1, 2), max_features=int(length/10)) 
x = cv.fit_transform(corpus).toarray() #converting to array
y = data['Habituality'] #dependent variable


## y is a categorical variable so will encode it
#le = LabelEncoder()
#y = le.fit_transform(y)

## now splitting the model into train and test set

#from sklearn.model_selection import train_test_split
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)



In [ ]:
## training the model
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB() # using naive bayes classification algorithm 
predict_fit = model.fit(x,y)# fitting the model

kfold = StratifiedKFold(n_splits=10)
predictions = cross_val_predict(model, x, y, cv=kfold)
target_names = ['nonhabitual', 'habitual']

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, predictions)


print(cm)

In [ ]:
#print(sen)

In [ ]:
print(classification_report(y, predictions, target_names=target_names))

In [ ]:
print(data)

In [ ]:
print(len(predictions))

In [ ]:
beNum = []
for Sentence in sen:
    numBe = 0
    if nlp(Sentence.text) != parsed:
        parsed = nlp(Sentence.text)
        for i, word in enumerate(parsed):
            if (word.text.lower() == "be"):
                numBe += 1
    beNum.append([Sentence.text,numBe])
    

In [ ]:
for i in beNum:
    print(i)

In [ ]:
print(len(beNum))

In [ ]:
print(parsed)
print(nlp(Sentence.text))

In [ ]:
for i in extraBe:
    print (i)

In [ ]:
nltk.download('tagsets')

In [ ]:
from nltk.data import load
tagdict = load('help/tagsets/upenn_tagset.pickle')
#tagdict['NN'][0]
tagdict.keys()

In [ ]:
#mapping numerical values to various part of speech list
pos_list = pd.DataFrame({'pos': dir(spacy.parts_of_speech)})
#pos_list = copy.deepcopy(dir(spacy.parts_of_speech))
num = []
for i in range((len(pos_list))):
    num.append(i)
pos_list.insert(1, 'num', num)
print(pos_list)

In [ ]:
#mapping the same numerical values to the part of speech of te actual words in the sentences before and after 'be'
for i in range(len(w)):
    for j in range(len(pos_list)):
        if (w[i] == pos_list.iloc[j, 0]):
            w[i] = pos_list.iloc[j, 1]
        if (pppp[i] == pos_list.iloc[j, 0]):
            pppp[i] = pos_list.iloc[j, 1]
        if (ppp[i] == pos_list.iloc[j, 0]):
            ppp[i] = pos_list.iloc[j, 1]
        if (pp[i] == pos_list.iloc[j, 0]):
            pp[i] = pos_list.iloc[j, 1]
        if (p[i] == pos_list.iloc[j, 0]):
            p[i] = pos_list.iloc[j, 1]
        if (a[i] == pos_list.iloc[j, 0]):
            a[i] = pos_list.iloc[j, 1]
        if (pa[i] == pos_list.iloc[j, 0]):
            pa[i] = pos_list.iloc[j, 1]
        if (ppa[i] == pos_list.iloc[j, 0]):
            ppa[i] = pos_list.iloc[j, 1]
        if (pppa[i] == pos_list.iloc[j, 0]):
            pppa[i] = pos_list.iloc[j, 1]

In [ ]:
#data = data.drop(columns=['n-gram'], axis = 1)
#data = data.drop(columns=['word', 'preprepreprevious', 'prepreprevious', 'preprevious', 'previous','after','postafter', 'postpostafter', 'postpostpostafter'], axis=1)

In [ ]:
#creating the data that has the  n-gram,and all the part of speech mappings
data.insert(len(data.columns), "n-gram", predictions)
data.insert(len(data.columns), "word", w)
data.insert(len(data.columns), "preprepreprevious", pppp)
data.insert(len(data.columns), "prepreprevious", ppp)
data.insert(len(data.columns), "preprevious", pp)
data.insert(len(data.columns), "previous", p)
data.insert(len(data.columns), "after", a)
data.insert(len(data.columns), "postafter", pa)
data.insert(len(data.columns), "postpostafter", ppa)
data.insert(len(data.columns), "postpostpostafter", pppa)

In [ ]:
X = data.drop(columns=['Habituality', 'Sentence'], axis=1)
y = data['Habituality']

In [ ]:
print(X)
print(y)

In [ ]:
#creating the pipeline model
scaler = StandardScaler()
lr = LogisticRegression(class_weight='balanced', max_iter = 500)
estimators=[('lr', LogisticRegression(max_iter = 10000, class_weight='balanced')), ('mlp', MLPClassifier(max_iter = 10000, solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)), ('svc', SVC(probability=True, max_iter = 10000, class_weight='balanced'))]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='soft')
model4 = Pipeline([('standardize', scaler),
                    ('ensemble', ensemble)])

In [ ]:
parameters = {'C':('ensemble', 'rbf'), 'C': [1, 10]}
#model4 = GridSearchCV(lr, parameters, scoring='recall_weighted')


In [ ]:
kfold = StratifiedKFold(n_splits=10)
predictions = cross_val_predict(model4, X, y, cv=kfold)
target_names = ['nonhabitual', 'habitual']

In [ ]:

fold_actual = [y[j] for i, j in kfold.split(X,y)]
fold_pred = [predictions[j] for i, j in kfold.split(X,y)]
each_f = []
f_score_summary = []
for i in range(len(fold_actual)):
    fold_actual[i] = np.array(fold_actual[i])
    report = classification_report(fold_actual[i], fold_pred[i], target_names=target_names, output_dict=True)
    each_f.extend((report['nonhabitual']['f1-score'], report['habitual']['f1-score'], report['weighted avg']['f1-score']))
    f_score_summary.append(each_f)
    each_f = []


In [ ]:
z = data['Sentence']

In [ ]:
k_fold_train_i = pd.DataFrame()
k_fold_train_eo = pd.DataFrame()
k_fold_test_i = pd.DataFrame()
k_fold_test_eo = pd.DataFrame()
count = 0
for i, j in kfold.split(X,y):
    y_train, y_test = y[i], y[j]
    z_train, z_test = z[i], z[j]
    x_train, x_test = X.iloc[i], X.iloc[j]
    train_input_csv = 'Augmented Dependency Kfold Baseline 2' + str(count) + ' Train Input.csv'
    train_output_csv = 'Augmented Dependency Kfold Baseline 2' + str(count) + ' Train Expected Output.csv'
    test_input_csv = 'Augmented Dependency Kfold Baseline 2' + str(count) + ' Test Input.csv'
    test_output_csv = 'Augmented Dependency Kfold Baseline 2' + str(count) + ' Test Expected Output.csv'
    with open(train_input_csv, 'w+', newline='') as file:
       x_train.to_csv(train_input_csv)
    with open(train_output_csv, 'w+', newline='') as file:
       y_train.to_csv(train_output_csv)
    with open(test_input_csv, 'w+', newline='') as file:
       x_test.to_csv(test_input_csv)
    with open(test_output_csv, 'w+', newline='') as file:
       y_test.to_csv(test_output_csv)
    count += 1


#k_fold_train = pd.concat([k_fold_input, z[j]])
#OK ABOVE JUST GIVES TRAIN/TEST FOR ONE GOTTA DO IT FO RALL
    
#k_fold_input = pd.concat([k_fold_input, z[j]])
#k_fold_expected_output = pd.concat([k_fold_expected_output, y[j]])
    
    
#for train_index, test_index in skf.split(X, y):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

In [ ]:
#df = pd.DataFrame(f_score_summary)
#df.plot(kind='box')
non_hab = []
hab = []
avg = []
for i in range(len(f_score_summary)):
    non_hab.append(f_score_summary[i][0])
    hab.append(f_score_summary[i][1])
    avg.append(f_score_summary[i][2])
nh_max = max(non_hab)
h_max = max(hab)
wa_max = max(avg)
nh_min = min(non_hab)
h_min = min(hab)
wa_min = min(avg)
print(nh_max, h_max, wa_max, nh_min, h_min, wa_min)

In [ ]:
#f_score_summary

In [ ]:
confusion_matrix(y, predictions)

In [ ]:
print(classification_report(y, predictions, target_names=target_names))

In [ ]:
kfold = StratifiedKFold(n_splits=10)
prediction_probs = cross_val_predict(model4, X, y, cv=kfold, method = 'predict_proba')
prediction_new = copy.deepcopy(prediction_probs)
prediction_new = prediction_new[:,0]
for i in range(len(prediction_probs)):
    # 0.79
    if (prediction_new[i] >= 0.84):#0.64 #was 0.94
        prediction_new[i] = -1
    else:
        prediction_new[i] = 1
#change value if number rules changed!!!!
#for i in range(len(prediction_probs)):
#    if (data_new.iloc[i, 13] == 1 and data_new.iloc[i, 15] == 1):
#        prediction_new[i] = 1
        
data_new = copy.deepcopy(data)
data_new.insert(1, 'Probability Non-Habitual', prediction_probs[:,0])

In [ ]:
model_fit = model4.fit(X,y)

In [ ]:
#joblib.dump(model_fit, "harrison habituality_model.joblib")

In [ ]:
#with open('new predicted coraal_analysis_spreadsheet.csv', 'w+', newline='') as file:
#   data_new.to_csv('new predicted coraal_analysis_spreadsheet.csv')

In [ ]:
#for i in range(len(data_new)):
#   data_new.iat[i,0] = prediction_new[i]

In [ ]:
confusion_matrix(y, prediction_new)

In [ ]:
print(classification_report(y, prediction_new, target_names=target_names))

In [ ]:
#falsePredictedNewData = copy.deepcopy(data_new)
#newPredictedNewData = copy.deepcopy(data_new)
#posPredictedNewData = copy.deepcopy(data_new)
#i = len(falsePredictedNewData)-1
#while (i >= 0):
#    falsePredictedNewData.drop(i, inplace=True)
#    posPredictedNewData.drop(i, inplace=True)
#    i = i -1
#for i in range(len(data_new)):
#    if (data_new.loc[i, 'Habituality'] != data.loc[i, 'Habituality']):
#        falsePredictedNewData = falsePredictedNewData.append(data_new.iloc[i], ignore_index=True)
#    else:
#        posPredictedNewData = posPredictedNewData.append(data_new.iloc[i], ignore_index=True)

In [ ]:
#with open('false predicted new coraal_analysis_spreadsheet.csv', 'w+', newline='') as file:
#   falsePredictedNewData.to_csv('false predicted new coraal_analysis_spreadsheet.csv')#